### 1.什么是ReAct Agent
ReAct Agent是一种目前非常流行的Agent框架，核心思想是让模型在解决问题时动态交替进行逻辑推理和环境交互，模仿人类的决策过程。
ReAct Agent的核心工作流程包括以下三个步骤：
1. Reasoning（推理）：LLM分析用户的输入，思考需要做什么，决定是否需要调用工具来解决问题。
2. Acting（行动）：如果需要调用工具，LLM会根据推理结果调用相应的工具，并将工具的输出作为环境反馈。
3. Observation（观察）：LLM会根据工具的输出，更新自己的内部状态，继续推理和行动，直到问题解决。
这三个步骤形成一个循环：推理 -> 行动 -> 观察 -> 推理 ...直到LLM认为已经收集到足够的信息，可以给出最终答案为止。
#### 为什么需要ReAct Agent
ReAct模式的出现解决了传统AI Agent的几个关键问题：
1. 能力边界问题：纯LLM虽然知识丰富，但无法获取实时信息、执行具体操作或访问外部系统。ReAct模式能够通过工具调用扩展LLM的能力边界。
2. 决策智能性：ReAct模式能够让Agent能够根据具体情境去智能判断是否需要调用工具，以及调用哪个工具。
3. 复杂问题处理：对于需要多步骤推理的复杂问题，ReAct的循环机制允许Agent进行多次推理-行动-观察，逐步收集信息并完善答案。
### 2.工具调用
工具调用，是一种让AI Agent能够使用外部工具的机制。
某种意义上，Tools跟Message、Template一样，本质上都是根据不同的目的，对向模型输入的信息进行标准化、规范化的方式。
在LangGraph中，工具调用的流程通常为：
- LLM分析用户输入，决定是否需要调用工具；
- 如果需要，LLM会生成工具调用请求（tool_calls）；
- 工具节点（ToolNode）执行工具调用；
- 工具执行结果返回给LLM；
- LLM根据工具结果生成最终回复；
### 3.子图
子图（Subgraph）是指在LangGraph中，将一个复杂的任务分解为多个子任务，每个子任务对应一个子图。
每个子图都是一个独立的Agent，负责完成一个具体的子任务。
子图之间可以通过条件边（Conditional Edge）进行连接，根据子任务的执行结果，动态选择下一个子任务。
子图本身是一个独立的Graph，但它又能作为一个节点（Node）嵌入到另一个Graph（父图）中。
子图的优势在于：
1. 任务分解：将复杂任务分解为多个简单任务，每个任务由一个子图负责，使任务管理更加清晰。
2. 并行执行：多个子图可以并行执行，提高任务处理效率。
3. 可维护性：每个子图都是一个独立的模块，易于维护和调试。
### 4.简易ReAct Agent的代码实现
1. 一个子图，负责处理工具调用循环；
2. 一个主图，负责管理整个对话循环；


In [ ]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, tool
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from datetime import datetime

load_dotenv()

# 子图的状态
class SubAgentState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

llm = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    temperature=0.5,
)

# 创建工具
@tool
def get_current_time() -> str:
    """获取当前的日期和时间，返回格式化的时间字符串。当用户询问时间相关问题时使用此工具。"""
    now = datetime.now()
    # 格式化日期和时间
    date_str = now.strftime("%Y年%m月%d日")
    time_str = now.strftime("%H:%M:%S")
    # 获取星期几
    weekdays = ["星期一", "星期二", "星期三", "星期四", "星期五", "星期六", "星期日"]
    weekday_str = weekdays[now.weekday()]
    # 判断上午/下午
    hour = now.hour
    if hour < 12:
        period = "上午"
    elif hour < 18:
        period = "下午"
    else:
        period = "晚上"
    # 返回友好的时间描述
    return f"当前时间是：{date_str} {weekday_str} {period} {time_str}"

